In [1]:
!pip install scikit-learn


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.7 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.7 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.7 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.7 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.7 MB ? eta -:--:--
   -- ------------------------------------- 0.5/8.7 MB 239.8 kB/s eta 0:00:35
   --- ------------------------------------ 0.8/8.7 MB 377.2 kB/s eta 0:00:22
   --- ------------------------------------ 0.8/8.7 MB 377.2 kB/s eta 0:00:22
   --- ------------------------------------ 0.8/8.7 MB 377.2 kB/s eta 0:00:22
   --- ------------------------------------ 0.8/8.7 MB 377.2 kB/s eta 0:00:22
   --- ------------------------------------ 0.8/8


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Basic imports and versions (run this first)
import sys
import pandas as pd
import numpy as np
import sklearn
print("Python:", sys.version.splitlines()[0])
print("pandas:", pd.__version__)
print("scikit-learn:", sklearn.__version__)


Python: 3.12.4 (tags/v3.12.4:8e8a4ba, Jun  6 2024, 19:30:16) [MSC v.1940 64 bit (AMD64)]
pandas: 2.3.1
scikit-learn: 1.7.2


In [3]:
# Replace 'fruits.xlsx' with the uploaded filename if different
df = pd.read_excel('fruit_data.xlsx')
# Show the first 5 rows; .head() shows only the first 5 rows of DataFrame by default.
df.head()
# Check the number of rows and columns
# X.shape


,Unnamed: 0,fruit_type,color,size,weight
0,0,grape,Yellow,Tiny,8.303385
1,1,apple,Pink,Largee,80.976370
2,2,banana,Pale Yellow,Large,74.615192
3,3,grape,Red,Tiny,6.924070
4,4,banana,Creamy White,Largee,82.002542


In [4]:
# Quick checks
print("Rows, columns:", df.shape)
print("\nColumn types:\n")
print(df.dtypes)
print("\nMissing values per column:")
print(df.isna().sum())
# Summary statistics for numeric columns
df.describe()


Rows, columns: (200, 5)

Column types:

Unnamed: 0      int64
fruit_type     object
color          object
size           object
weight        float64
dtype: object

Missing values per column:
Unnamed: 0    0
fruit_type    0
color         0
size          0
weight        0
dtype: int64


,Unnamed: 0,weight
count,200.000000,200.000000
mean,99.500000,59.047220
std,57.879185,46.697739
min,0.000000,1.000000
25%,49.750000,8.142752
50%,99.500000,63.109415
75%,149.250000,94.367815
max,199.000000,250.000000


In [5]:
# Normalize textual columns (lowercase, strip spaces)
df['color'] = df['color'].astype(str).str.strip().str.lower()

# If "size" is text like 'small', 'medium', 'large', show unique values:
print("Unique sizes:", df['size'].unique())

# Remove exact duplicate rows (if any)
before = len(df)
df = df.drop_duplicates()
print("Dropped duplicates:", before - len(df))

# If weight has missing values, fill with median (safe simple choice)
if df['weight'].isna().sum() > 0:
    med = df['weight'].median()
    df['weight'] = df['weight'].fillna(med)
    print("Filled missing weight with median:", med)

# Show again
df.isna().sum()


Unique sizes: ['Tiny' 'Largee' 'Large' 'Small' 'Medium']
Dropped duplicates: 0


Unnamed: 0    0
fruit_type    0
color         0
size          0
weight        0
dtype: int64

In [6]:
# Clean up 'size' text
df['size'] = df['size'].astype(str).str.strip().str.lower()

# Fix the typo: "largee" → "large"
df['size'] = df['size'].replace({'largee': 'large'})

# Show unique sizes again
print("Cleaned unique sizes:", df['size'].unique())


Cleaned unique sizes: ['tiny' 'large' 'small' 'medium']


In [7]:
# Merge "tiny" into "small"
df['size'] = df['size'].replace({'tiny': 'small'})
print("Final size categories:", df['size'].unique())


Final size categories: ['small' 'large' 'medium']


In [8]:
# Average weight per fruit_type and size
pd.pivot_table(df, values='weight', index='fruit_type', columns='size', aggfunc='mean')


size,large,medium,small
fruit_type,,,
apple,89.162148,90.103062,105.211493
banana,74.199271,76.414899,74.593160
grape,NaN,6.289663,7.135950


**Data Consistency Check**

After calculating the average weight by fruit type and size, I noticed that the **size** and **weight** columns are not perfectly aligned.  

For example:  
- Small apples (e.g., line 18 in the Excel file) are sometimes heavier than large ones (e.g., line 1).  
- Grape entries have no “large” size values, and the average weight of “small” grapes is higher than that of “medium” grapes.  

These observations suggest that the size labels may have been defined independently within each fruit type, or were entered inconsistently across the dataset.  

For this project, I decided to **keep the data as-is**, but I note this inconsistency in my analysis. The model should still be able to learn useful patterns from the existing features.


In [9]:
# Step 6: Define target and features
y = df['fruit_type']                # this is what we want to predict
X = df.drop(columns=['fruit_type']) # everything else is used as input

# Quick check
print("Features (X):", X.columns.tolist())
print("Target (y): unique labels ->", y.unique())


Features (X): ['Unnamed: 0', 'color', 'size', 'weight']
Target (y): unique labels -> ['grape' 'apple' 'banana']


In [10]:
X.head()

,Unnamed: 0,color,size,weight
0,0,yellow,small,8.303385
1,1,pink,large,80.976370
2,2,pale yellow,large,74.615192
3,3,red,small,6.924070
4,4,creamy white,large,82.002542


In [11]:
# Convert size to numeric scale 7
size_map = {'small': 0, 'medium': 1, 'large': 2}
X['size_num'] = X['size'].map(size_map)
X = X.drop(columns=['size'])  # remove old text column

# One-hot encode the color column
X = pd.get_dummies(X, columns=['color'], drop_first=True)

# Show first few rows
X.head()


,Unnamed: 0,weight,size_num,color_creamy white,color_green,color_pale yellow,color_pink,color_purple,color_red,color_yellow,color_yellow1
0,0,8.303385,0,False,False,False,False,False,False,True,False
1,1,80.976370,2,False,False,False,True,False,False,False,False
2,2,74.615192,2,False,False,True,False,False,False,False,False
3,3,6.924070,0,False,False,False,False,False,True,False,False
4,4,82.002542,2,True,False,False,False,False,False,False,False


In [12]:
# Convert all boolean columns to integers (0/1)
X = X.astype(int)

# Check the first few rows
X.head()


,Unnamed: 0,weight,size_num,color_creamy white,color_green,color_pale yellow,color_pink,color_purple,color_red,color_yellow,color_yellow1
0,0,8,0,0,0,0,0,0,0,1,0
1,1,80,2,0,0,0,1,0,0,0,0
2,2,74,2,0,0,1,0,0,0,0,0
3,3,6,0,0,0,0,0,0,1,0,0
4,4,82,2,1,0,0,0,0,0,0,0


In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)
print("Train size:", X_train.shape[0], "Test size:", X_test.shape[0])


Train size: 160 Test size: 40


In [14]:
# Model to classify fruits
from sklearn.linear_model import LogisticRegression
# Metrics to evaluate how well the model performs
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Re-split the data (in case you modified X after the previous split)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Create and train the model
model = LogisticRegression(max_iter=200, random_state=42)
model.fit(X_train, y_train)

# Predict fruit types for the test data; stored in y_pred
y_pred = model.predict(X_test)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))                            # Percentage of correctly classified fruits
print("F1 (macro):", f1_score(y_test, y_pred, average='macro'))               # Balances precision and recall across all three fruit classes
print("\nClassification report:\n", classification_report(y_test, y_pred))    # Gives a detailed report including precision, recall, F1 for each fruit

# Support = number of samples of each fruit in the test set.

Accuracy: 0.75
F1 (macro): 0.7483380816714149

Classification report:
               precision    recall  f1-score   support

       apple       0.67      0.57      0.62        14
      banana       0.64      0.69      0.67        13
       grape       0.93      1.00      0.96        13

    accuracy                           0.75        40
   macro avg       0.75      0.75      0.75        40
weighted avg       0.74      0.75      0.75        40



C:\Users\JJE4FE\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 200 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=200).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [15]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Re-train logistic regression with scaled data
model = LogisticRegression(max_iter=500, random_state=42)
model.fit(X_train_scaled, y_train)
y_pred_scaled = model.predict(X_test_scaled)

# Evaluate again
from sklearn.metrics import accuracy_score, f1_score, classification_report
print("Accuracy (scaled):", accuracy_score(y_test, y_pred_scaled))
print("F1 (macro, scaled):", f1_score(y_test, y_pred_scaled, average='macro'))
print("\nClassification Report:\n", classification_report(y_test, y_pred_scaled))


Accuracy (scaled): 0.775
F1 (macro, scaled): 0.7739791073124406

Classification Report:
               precision    recall  f1-score   support

       apple       0.69      0.64      0.67        14
      banana       0.69      0.69      0.69        13
       grape       0.93      1.00      0.96        13

    accuracy                           0.78        40
   macro avg       0.77      0.78      0.77        40
weighted avg       0.77      0.78      0.77        40



**Logistic Regression Model**

We trained a Logistic Regression model to classify the fruits (`apple`, `banana`, `grape`) based on their features (`weight`, `size_num`, and one-hot encoded colors).

**Initial Results**

- **Accuracy:** 0.75  
- **F1 score (macro):** 0.7483380816714149  

**Classification Report:**

| Fruit   | Precision | Recall | F1-score | Support |
|---------|-----------|--------|----------|---------|
| apple   | 0.67      | 0.57   | 0.62     | 14      |
| banana  | 0.64      | 0.69   | 0.67     | 13      |
| grape   | 0.93      | 1.00   | 0.96     | 13      |

**Observations:**
- Grapes are classified very accurately.  
- Apples and bananas are sometimes confused due to overlapping feature values.  

**Convergence Warning**

During training, a warning appeared:

Warning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.


This occurred because the features (`weight`, `size_num`, color columns) are on very different scales. Logistic Regression performs better when numeric features are scaled to similar ranges.

**Solution: Feature Scaling**

We applied standard scaling (`StandardScaler`) to the numeric features (`weight`, `size_num`, and one-hot color columns), and increased `max_iter` to 500.  

After scaling, the model is expected to converge properly, and the performance metrics may improve slightly.


**Conclusion:** Logistic Regression can classify the fruits reasonably well, but overlapping feature values for apples and bananas limit performance.


In [16]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(random_state=42, max_depth=5)  # max_depth to avoid overfitting
tree.fit(X_train, y_train)
y_pred_tree = tree.predict(X_test)

print("Decision Tree Accuracy:", accuracy_score(y_test, y_pred_tree))
print("Decision Tree F1 (macro):", f1_score(y_test, y_pred_tree, average='macro'))
print("\nDecision Tree report:\n", classification_report(y_test, y_pred_tree))


Decision Tree Accuracy: 0.775
Decision Tree F1 (macro): 0.7739791073124406

Decision Tree report:
               precision    recall  f1-score   support

       apple       0.69      0.64      0.67        14
      banana       0.69      0.69      0.69        13
       grape       0.93      1.00      0.96        13

    accuracy                           0.78        40
   macro avg       0.77      0.78      0.77        40
weighted avg       0.77      0.78      0.77        40



In [17]:
# 11) Simple visualization: confusion matrix
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

ConfusionMatrixDisplay.from_estimator(model, X_test, y_test)
plt.title("Logistic Regression Confusion Matrix")
plt.show()

ConfusionMatrixDisplay.from_estimator(tree, X_test, y_test)
plt.title("Decision Tree Confusion Matrix")
plt.show()


ModuleNotFoundError: No module named 'matplotlib'